In [ ]:
# Import the necessary libraries (e.g., pandas, numpy, sklearn)
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the training dataset players_21
players_21_train= pd.read_csv("/content/drive/MyDrive/DreamTeamAi|JR /archive/players_21.csv")
players_22_test=pd.read_csv("/content/drive/MyDrive/DreamTeamAi|JR /archive/players_22.csv")

Drop 30%

In [ ]:
limit = 30

percentage = (players_21_train.isnull().sum()/len(players_21_train))*100

drop = percentage[percentage > limit].index
players_21_train.drop(columns = drop, inplace=True)

Filter objects that should be numeric and converting them to type int

In [ ]:
filtered_obj = players_21_train.filter(items=['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
                                    'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb',
                                    'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'])
def convert_to_number(entry):
    if '+' in entry:
        parts = entry.split('+')
        return int(parts[0]) + int(parts[1])
    elif '-' in entry:
        parts = entry.split('-')
        return int(parts[0]) - int(parts[1])
    else:
        return int(entry)

filtered_obj = filtered_obj.applymap(convert_to_number)

In [ ]:
players_21_train.update(filtered_obj)
int_columns = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
                'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb',
                'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

players_21_train[int_columns] = players_21_train[int_columns].astype(int)

In [ ]:
players_21_train.info(verbose=True)

Split Data into numbers and objects to impute numbers and fill then encode objects.

In [ ]:
numbers = players_21_train.select_dtypes(include=["number"])
objects = players_21_train.select_dtypes(exclude=["number"])

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
imputer.fit(numbers)
imputer_data = imputer.fit_transform(numbers)
numbers = pd.DataFrame(imputer_data, columns=numbers.columns)

In [ ]:
objects = objects.fillna(method='ffill', axis=0)

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
label = LabelEncoder()

for i in objects.columns:
  objects[i] = label.fit_transform(objects[i])

# Feature Selection

Combine objects and numbers back into a single dataframe

In [ ]:
players_21_train_combined = pd.concat([numbers, objects], axis=1)
players_21_train_combined.head()

Correlation Matrix to pick suitable features with correlation over 0.55

In [ ]:
corr_matrix = players_21_train_combined.corr()
pd.set_option('display.max_rows', None)
corr_matrix["overall"].sort_values()

In [ ]:
limit = 0.55
correlations = corr_matrix[corr_matrix['overall']>limit].index
selected_correlations = players_21_train_combined[correlations]
selected_correlations.info()

Splitting our data into features and target

In [ ]:
# SPLITTING
from sklearn.model_selection import train_test_split

y = selected_correlations["overall"]
X = selected_correlations.drop(["overall"], axis=1)


Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Scale the numeric columns
scaled_features = scaler.fit(X)

X = pd.DataFrame(X, columns=X.columns)

X_train,  X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# selected_correlations.head()

In [ ]:
import pickle
filename = 'scaler.pkl'
pickle.dump(scaler, open(filename, 'wb'))

Ensemble Model Training

In [ ]:
# MODEL  1      -  RANDOM   FOREST   REGRESSOR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)


# Train the model on the training data
rf_model.fit(X_train, y_train)


# Predictions on the validation set
predictions = rf_model.predict(X_val)

# Evaluate the model
mae = mean_absolute_error(y_val, predictions)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
r2 = r2_score(y_val, predictions)


print(f"Mean Absolute Error: {mae:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

In [ ]:
# MODEL  2      -     GRADIENT   BOOSTING   REGRESSOR
from sklearn.ensemble import GradientBoostingRegressor


# Initialize the model
gb_model = GradientBoostingRegressor(random_state=42)


# Train the model on the training data
gb_model.fit(X_train, y_train)


# Predictions on the validation set
predictions = gb_model.predict(X_val)

# Evaluate the model
mae = mean_absolute_error(y_val, predictions)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
r2 = r2_score(y_val, predictions)


print(f"Mean Absolute Error: {mae:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

In [ ]:
# MODEL  3 - XG   BOOST   REGRESSOR

from xgboost import XGBRegressor

# Initialize the model
xgb_model = XGBRegressor(random_state=42)

# Train the model on the training data
xgb_model.fit(X_train, y_train)


# Predictions on the validation set
predictions = xgb_model.predict(X_val)

# Evaluate the model
mae = mean_absolute_error(y_val, predictions)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
r2 = r2_score(y_val, predictions)


print(f"Mean Absolute Error: {mae:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")




In [ ]:
# MODEL  4 - ADA   BOOST  REGRESSOR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize the AdaBoost Regressor model
adb_model = AdaBoostRegressor(random_state=42)

# Train the model on the training data
adb_model.fit(X_train, y_train)

# Predictions on the validation set
predictions = adb_model.predict(X_val)

# Evaluate the model
mae = mean_absolute_error(y_val, predictions)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
r2 = r2_score(y_val, predictions)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")


## Cross Search Validation for best ensemble learning model

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True],
    'random_state': [0, 45],
    'n_jobs': [-1]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create GridSearchCV object for random forest model
rf_grid_search = GridSearchCV(rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Perform grid search on the data (X_train, y_train)
rf_grid_search.fit(X_train, y_train)


In [ ]:
# Check for best model
best_model = rf_grid_search.best_estimator_
best_model

In [ ]:
best_params = rf_grid_search.best_params_
best_params

In [ ]:
y_pred = rf_grid_search.predict(X_val)
mean_absolute_error(y_pred, y_val)

In [ ]:
rf_best_params = rf_grid_search.best_params_
print("Best Hyperparameters:", rf_best_params)

# SAVING MODEL AS FILE


In [ ]:
import pickle
filename = 'model.pkl'
pickle.dump(rf_model, open(filename, 'wb'))

# CLEANING TEST DATASET (2022 data)

In [ ]:
selected_columns_test = ['overall','potential','value_eur','wage_eur','release_clause_eur','power_shot_power','passing','dribbling','movement_reactions','mentality_composure']
test_columns = players_22_test[selected_columns_test]

In [ ]:
imputer.fit(test_columns)
imputed_data = imputer.fit_transform(test_columns)
test_data = pd.DataFrame(imputed_data, columns=test_columns.columns)

In [ ]:
yt = test_data["overall"]
Xt = test_data.drop(["overall"], axis=1)

In [ ]:
scaled_feature_test = scaler.fit_transform(Xt)
scaled_test = pd.DataFrame(Xt, columns=Xt.columns)
scaled_test.head()

In [ ]:
#Train Split
Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt, yt, test_size=0.2, random_state=42)

In [ ]:
rf_model.fit(Xt_train, yt_train)


# Predictions on the validation set
predictions_22 = rf_model.predict(Xt_test)

# Evaluate the model
mae = mean_absolute_error(yt_test, predictions_22)
rmse = np.sqrt(mean_squared_error(yt_test, predictions_22))
r2 = r2_score(yt_test, predictions_22)


print(f"Mean Absolute Error: {mae:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

In [ ]:
values = {"Ytest": yt_test, "Ypred": predictions_22}
df = pd.DataFrame(values)

df.to_csv('ytest-ypred.csv', index=False)

# Deployment in a different file